### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/303_TROCAS7_Fusion_Apr2021_DENOVO_157/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/303_TROCAS7_Fusion_Apr2021_DENOVO_157


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks302 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/303_TROCAS7_Fusion_Apr2021_DENOVO_157/303_TROCAS7_Fusion_Apr2021_DENOVO_157_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks302))

print(peaks302.columns)

# These columns mess things up- get rid of them

del peaks302['Fraction']
del peaks302['Scan']
del peaks302['Source File']
del peaks302['Tag Length']
del peaks302['PTM']
del peaks302['tag (>=0%)']
del peaks302['mode']
del peaks302['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks302.columns = columns

mean_len = peaks302['length'].mean()
print(mean_len)

# look at the dataframe
peaks302.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 275
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.341818181818182


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2874,2,47.03,62400000.0,1044.5564,3.7
1,REKM(+15.99)SSR,98,7,455.2310,2,30.48,346000.0,908.4498,-2.6
2,QFSSSYLSR,97,9,537.7650,2,64.23,226000.0,1073.5142,1.2
3,VVTVSLPR,97,8,435.7753,2,64.26,95700000.0,869.5334,3.0
4,SLVNLNSK,95,8,437.7535,2,61.01,813000.0,873.4919,0.6


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks302['A'] = peaks302['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks302['C'] = peaks302['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks302['D'] = peaks302['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks302['E'] = peaks302['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks302['F'] = peaks302['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks302['G'] = peaks302['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks302['H'] = peaks302['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks302 output, there will be no isoleucines (they're lumped in with leucines)
peaks302['I'] = peaks302['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks302['K'] = peaks302['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks302['L'] = peaks302['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks302['M'] = peaks302['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks302['N'] = peaks302['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks302['P'] = peaks302['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks302['Q'] = peaks302['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks302['R'] = peaks302['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks302['S'] = peaks302['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks302['T'] = peaks302['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks302['V'] = peaks302['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks302['W'] = peaks302['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks302['Y'] = peaks302['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks302['c-carb'] = peaks302['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks302['m-oxid'] = peaks302['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks302['n-deam'] = peaks302['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks302['q-deam'] = peaks302['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks302['stripped_peptide'] = peaks302['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks302['stripped_length'] = peaks302['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks302['ptm-total'] = peaks302['c-carb'] + peaks302['m-oxid'] + peaks302['n-deam'] + peaks302['q-deam']

# calculate NAAF numerator for each peptide k
peaks302['NAAF_num.'] = peaks302['Area'] / peaks302['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50.csv")

# check out the results
peaks302.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2874,2,47.03,62400000.0,1044.5564,3.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,6.240000e+06
1,REKM(+15.99)SSR,98,7,455.2310,2,30.48,346000.0,908.4498,-2.6,0,...,0,0,0,1,0,0,REKMSSR,7,1,4.942857e+04
2,QFSSSYLSR,97,9,537.7650,2,64.23,226000.0,1073.5142,1.2,0,...,0,1,0,0,0,0,QFSSSYLSR,9,0,2.511111e+04
3,VVTVSLPR,97,8,435.7753,2,64.26,95700000.0,869.5334,3.0,0,...,0,0,0,0,0,0,VVTVSLPR,8,0,1.196250e+07
4,SLVNLNSK,95,8,437.7535,2,61.01,813000.0,873.4919,0.6,0,...,0,0,0,0,0,0,SLVNLNSK,8,0,1.016250e+05


In [5]:
# keep only stripped peptide I/L and NAAF
dn_302 = peaks302[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_302.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/302C_BY_T24_GF_DN50_ILnaafs.csv")

dn_302.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,62400000.0,6.240000e+06
1,REKMSSR,346000.0,4.942857e+04
2,QFSSSYLSR,226000.0,2.511111e+04
3,VVTVSLPR,95700000.0,1.196250e+07
4,SLVNLNSK,813000.0,1.016250e+05


In [6]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks302['A'].sum(),
        'C': peaks302['C'].sum(),
        'D': peaks302['D'].sum(),
        'E': peaks302['E'].sum(),
        'F': peaks302['F'].sum(),
        'G': peaks302['G'].sum(),
        'H': peaks302['H'].sum(),
        'I': peaks302['I'].sum(),
        'K': peaks302['K'].sum(),
        'L': peaks302['L'].sum(),
        'M': peaks302['M'].sum(),
        'N': peaks302['N'].sum(),
        'P': peaks302['P'].sum(),
        'Q': peaks302['Q'].sum(),
        'R': peaks302['R'].sum(),
        'S': peaks302['S'].sum(),
        'T': peaks302['T'].sum(),
        'V': peaks302['V'].sum(),
        'W': peaks302['W'].sum(),
        'Y': peaks302['Y'].sum(),
        'c-carb': peaks302['c-carb'].sum(),
        'm-oxid': peaks302['m-oxid'].sum(),
        'n-deam': peaks302['n-deam'].sum(),
        'q-deam': peaks302['q-deam'].sum(),
        'Total area': peaks302['Area'].sum(),
        'Total length': peaks302['stripped_length'].sum()
       }

totalpeaks302 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks302['% C w/ carb'] = totalpeaks302['c-carb'] / totalpeaks302['C'] 

# calculate percentage of M's that are oxidized
totalpeaks302['% M w/ oxid'] = totalpeaks302['m-oxid'] / totalpeaks302['M'] 

# calculate percentage of N's that are deamidated
totalpeaks302['% N w/ deam'] = totalpeaks302['n-deam'] / totalpeaks302['N'] 

# calculate percentage of N's that are deamidated
totalpeaks302['% Q w/ deam'] = totalpeaks302['q-deam'] / totalpeaks302['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks302['NAAF denom.'] = totalpeaks302['Total area'] / totalpeaks302['Total length']

# write modified dataframe to new txt file
totalpeaks302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_totals.csv")

totalpeaks302.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,173,55,48,80,114,148,257,0,237,345,...,37,17,3,1.273473e+09,2690,1.0,0.201087,0.166667,0.0625,473409.909071


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 473409.909071

# use NAAF >50% ALC to get NAAF factor
peaks302['NAAF factor'] = (peaks302['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_302 = peaks302[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_302['A-NAAF50'] = peaksAAPTM_302['A'] * peaks302['NAAF factor']
peaksAAPTM_302['C-NAAF50'] = peaksAAPTM_302['C'] * peaks302['NAAF factor']
peaksAAPTM_302['D-NAAF50'] = peaksAAPTM_302['D'] * peaks302['NAAF factor']
peaksAAPTM_302['E-NAAF50'] = peaksAAPTM_302['E'] * peaks302['NAAF factor']
peaksAAPTM_302['F-NAAF50'] = peaksAAPTM_302['F'] * peaks302['NAAF factor']
peaksAAPTM_302['G-NAAF50'] = peaksAAPTM_302['G'] * peaks302['NAAF factor']
peaksAAPTM_302['H-NAAF50'] = peaksAAPTM_302['H'] * peaks302['NAAF factor']
peaksAAPTM_302['I-NAAF50'] = peaksAAPTM_302['I'] * peaks302['NAAF factor']
peaksAAPTM_302['K-NAAF50'] = peaksAAPTM_302['K'] * peaks302['NAAF factor']
peaksAAPTM_302['L-NAAF50'] = peaksAAPTM_302['L'] * peaks302['NAAF factor']
peaksAAPTM_302['M-NAAF50'] = peaksAAPTM_302['M'] * peaks302['NAAF factor']
peaksAAPTM_302['N-NAAF50'] = peaksAAPTM_302['N'] * peaks302['NAAF factor']
peaksAAPTM_302['P-NAAF50'] = peaksAAPTM_302['P'] * peaks302['NAAF factor']
peaksAAPTM_302['Q-NAAF50'] = peaksAAPTM_302['Q'] * peaks302['NAAF factor']
peaksAAPTM_302['R-NAAF50'] = peaksAAPTM_302['R'] * peaks302['NAAF factor']
peaksAAPTM_302['S-NAAF50'] = peaksAAPTM_302['S'] * peaks302['NAAF factor']
peaksAAPTM_302['T-NAAF50'] = peaksAAPTM_302['T'] * peaks302['NAAF factor']
peaksAAPTM_302['V-NAAF50'] = peaksAAPTM_302['V'] * peaks302['NAAF factor']
peaksAAPTM_302['W-NAAF50'] = peaksAAPTM_302['W'] * peaks302['NAAF factor']
peaksAAPTM_302['Y-NAAF50'] = peaksAAPTM_302['Y'] * peaks302['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_302['ccarb-NAAF50'] = peaksAAPTM_302['c-carb'] * peaksAAPTM_302['NAAF factor']
peaksAAPTM_302['moxid-NAAF50'] = peaksAAPTM_302['m-oxid'] * peaksAAPTM_302['NAAF factor']
peaksAAPTM_302['ndeam-NAAF50'] = peaksAAPTM_302['n-deam'] * peaksAAPTM_302['NAAF factor']
peaksAAPTM_302['qdeam-NAAF50'] = peaksAAPTM_302['q-deam'] * peaksAAPTM_302['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_naaf.csv")

peaksAAPTM_302.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,13.180966,1,0,0,0,0,0,0,0,...,13.180966,39.542899,13.180966,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
1,REKMSSR,0.104410,0,0,0,1,0,0,0,1,...,0.208819,0.208819,0.000000,0.000000,0.0,0.000000,0.0,0.10441,0.0,0.0
2,QFSSSYLSR,0.053043,0,0,0,0,1,0,0,0,...,0.053043,0.212172,0.000000,0.000000,0.0,0.053043,0.0,0.00000,0.0,0.0
3,VVTVSLPR,25.268799,0,0,0,0,0,0,0,0,...,25.268799,25.268799,25.268799,75.806398,0.0,0.000000,0.0,0.00000,0.0,0.0
4,SLVNLNSK,0.214666,0,0,0,0,0,0,0,1,...,0.000000,0.429332,0.000000,0.214666,0.0,0.000000,0.0,0.00000,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_302['NAAF factor'].sum(),
        'A': peaksAAPTM_302['A-NAAF50'].sum(),
        'C': peaksAAPTM_302['C-NAAF50'].sum(),
        'D': peaksAAPTM_302['D-NAAF50'].sum(),
        'E': peaksAAPTM_302['E-NAAF50'].sum(),
        'F': peaksAAPTM_302['F-NAAF50'].sum(),
        'G': peaksAAPTM_302['G-NAAF50'].sum(),
        'H': peaksAAPTM_302['H-NAAF50'].sum(),
        'I': peaksAAPTM_302['I-NAAF50'].sum(),
        'K': peaksAAPTM_302['K-NAAF50'].sum(),
        'L': peaksAAPTM_302['L-NAAF50'].sum(),
        'M': peaksAAPTM_302['M-NAAF50'].sum(),
        'N': peaksAAPTM_302['N-NAAF50'].sum(),
        'P': peaksAAPTM_302['P-NAAF50'].sum(),
        'Q': peaksAAPTM_302['Q-NAAF50'].sum(),
        'R': peaksAAPTM_302['R-NAAF50'].sum(),
        'S': peaksAAPTM_302['S-NAAF50'].sum(),
        'T': peaksAAPTM_302['T-NAAF50'].sum(),
        'V': peaksAAPTM_302['V-NAAF50'].sum(),
        'W': peaksAAPTM_302['W-NAAF50'].sum(),
        'Y': peaksAAPTM_302['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_302['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_302['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_302['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_302['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 473409.909071

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,326.318994,245.406633,14.827091,6.841099,7.079987,7.931387,22.073525,13.214814,0.0,14.306784,...,5.920139,14.827091,3.069664,0.720496,0.133664,1.0,0.312962,0.019807,0.046795,0.000689


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks302[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/302_BY_T24_GF_DN/302C_BY_T24_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks302['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks302[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 275
# nonredundant stripped Peaks peptides >50% ALC 180
average peptide length Peaks peptides >50% ALC 9.781818181818181
# redundant Peaks peptides >50% ALC 275
# nonredundant Peaks peptides 181


,stripped_peptide
0,LSSPATLNSR
1,REKMSSR
2,QFSSSYLSR
3,VVTVSLPR
4,SLVNLNSK
